Building end to end LLM Project that covers a real life industry use-case of equity research analysis.
We are building a news research tool where you can give bunch of news article URL and when you ask a question, it will retreive answers based on those URL.

In [24]:
#textloader allows to laod data from text file
from langchain.document_loaders import TextLoader


In [26]:
# Specify the path to your text file
file_path = r"C:\LLM_Project\News_1.txt"

# Create a TextLoader instance with the file path
loader = TextLoader(file_path)

# Load the data from the file
data = loader.load()

# Print the loaded data
print(data[0].page_content) #it shows the main content of our file
print(data[0].metadata) # meta data is name of our text file.

The stock of NVIDIA Corp (NASDAQ:NVDA) experienced a daily loss of -3.56% and a 3-month gain of 32.35%. With an Earnings Per Share (EPS) (EPS) of $1.92, the question arises: is the stock significantly overvalued? This article aims to provide a detailed valuation analysis of NVIDIA, offering insights into its financial strength, profitability, growth, and more. We invite you to delve into this comprehensive analysis.

Company Overview
Warning! GuruFocus has detected 10 Warning Signs with NVDA. Click here to check it out.

NVDA 30-Year Financial Data

The intrinsic value of NVDA


NVIDIA Corp (NASDAQ:NVDA) is a leading designer of discrete graphics processing units that enhance the experience on computing platforms. The firm's chips are widely used in various end markets, including PC gaming and data centers. In recent years, NVIDIA has broadened its focus from traditional PC graphics applications such as gaming to more complex and favorable opportunities, including artificial intelligen

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("movies.csv",source_column="title")
data = loader.load()
len(data)

9

In [4]:
print(type(data[0]))
data[0].page_content #content of your entire csv file/page

<class 'langchain_core.documents.base.Document'>


'movie_id: 101\ntitle: K.G.F: Chapter 2\nindustry: Bollywood\nrelease_year: 2022\nimdb_rating: 8.4\nstudio: Hombale Films\nlanguage_id: 3\nbudget: 1\nrevenue: 12.5\nunit: Billions\ncurrency: INR'

<!-- we are using Meta data in our project. So bassically, meta data will provide the  source link. -->

In [27]:
data[0].metadata

{'source': 'C:\\LLM_Project\\News_1.txt'}

In [31]:
#pip3 install unstructured libmagic python-magic python-magic-bin

#the unstructured library will kind of go to that particular website, we will have mentioned, will look to the dom structure/
#html file, and will pull all the information.

#so below we had given arguments as url,,,,

<!-- pip3 install unstructured libmagic python-magic python-magic-bin -->

In [6]:
loader = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/india/stockpricequote/refineries/relianceindustries/RI",
     "https://www.moneycontrol.com/us-markets/stockpricequote/apple/AAPL"
])

In [7]:
data = loader.load()
len(data)

Error fetching or processing https://www.moneycontrol.com/india/stockpricequote/refineries/relianceindustries/RI, exception: HTTPSConnectionPool(host='www.moneycontrol.com', port=443): Max retries exceeded with url: /india/stockpricequote/refineries/relianceindustries/RI (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))
Error fetching or processing https://www.moneycontrol.com/us-markets/stockpricequote/apple/AAPL, exception: HTTPSConnectionPool(host='www.moneycontrol.com', port=443): Max retries exceeded with url: /us-markets/stockpricequote/apple/AAPL (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))


0

So , we have huge block of text, we will divide it into chunks, and will sort the relevant chunks. lastly marging all this chunks together as a answer

In [9]:
# Taking some random text from wikipedia

text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. 
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. 
Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. 
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. 
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. 
Interstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.

Interstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014. 
It has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

In [10]:
# Say LLM token limit is 100, in that case we can do simple thing such as this

text[0:100]

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher N'

In [11]:
# Well but we want complete words and want to do this for entire text, may be we can use Python's split funciton

words = text.split(" ")
len(words)

264

In [12]:
chunks = []

s = ""
for word in words:
    s += word + " "
    if len(s)>200:
        chunks.append(s)
        s = ""
        
chunks.append(s)

In [13]:
chunks[:2]


['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt ',
 'Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in ']

In [14]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=200,
    chunk_overlap=0
)

In [15]:
chunks = splitter.split_text(text)
len(chunks)

Created a chunk of size 210, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 358, which is longer than the specified 200


9

In [16]:
for chunk in chunks:
    print(len(chunk))

105
120
210
181
197
207
128
357
253


In [17]:
text
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [18]:
chunks = r_splitter.split_text(text)

for chunk in chunks:
    print(len(chunk))

105
120
199
10
181
197
198
8
128
191
165
198
54


In [19]:
first_split = text.split("\n\n")[0]
first_split

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'

In [20]:
len(first_split)


439

In [21]:
second_split = first_split.split("\n")
second_split

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. ',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. ',
 'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.']

In [22]:
for split in second_split:
    print(len(split))

106
121
210


In [23]:
second_split[2]


'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'